## _Graph Segmentation_

1. _Connected Components Labelling (CCL)_
2. _Walkthrough Algorithm_
3. _Hierarchical Graph Networks_

In [ ]:
import glob, os, sys, yaml

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pprint
import seaborn as sns
import trackml.dataset

In [ ]:
import torch
from torch_geometric.data import Data
import itertools

In [ ]:
# append parent dir
sys.path.append('..')

In [ ]:
# get cuda gpus if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# local imports


### _Config File_

In [ ]:
# 
os.environ['EXATRKX_DATA'] = os.path.abspath(os.curdir)

In [ ]:
# load processing config file (trusted source)
config_path = 'LightningModules/Segmenting/configs/segment_quickstart.yaml'
config_file = os.path.join(os.curdir, config_path)
with open(config_file) as f:
    try:
        config = yaml.load(f, Loader=yaml.FullLoader) # equiv: yaml.full_load(f)
    except yaml.YAMLError as e:
        print(e)

In [ ]:
config["input_dir"] = 'run_all/gnn_processed/test'

### _Input Data_

In [ ]:
# fetch all files
inputdir = config["input_dir"]
gnn_files = sorted(glob.glob(os.path.join(inputdir, "*")))

In [ ]:
gnn_files[:5]

In [ ]:
gnn_data = torch.load(gnn_files[0], map_location=device)
print("Length of Data: {}".format(len(gnn_data)))

In [ ]:
gnn_data

## _CCL with Walkthrough_

- Run CCL Algorithm to Label Graphs
- Run Walkthrough Algorithm on Labelled Graphs


IDEA: Using CCL with low cut, one can force CCL not to break the component but rather keep the crossing tracks as single piece. Then build subgraphs from the connected-components and run walkthrough on these subgraphs. The subgraphs that are isolated tracks, the result will be trivial. For crossing tracks, Walkthrough should extract more tracks.

In [ ]:
import scipy.sparse as sps
import scipy.sparse.csgraph as scigraph
from torch_geometric.utils import to_scipy_sparse_matrix

In [ ]:
event_idx = 1
input_file = gnn_files[event_idx]

In [ ]:
graph = torch.load(input_file, map_location=device)
graph.scores = graph.scores[:graph.edge_index.shape[1]]  # resize to edge_index

In [ ]:
graph.edge_index.shape[1], graph.scores.shape[0]

In [ ]:
edge_cut = 0.25
e_mask = graph.scores > edge_cut

In [ ]:
new_graph = graph.clone()
new_graph.edge_index = new_graph.edge_index[:, e_mask]
new_graph.scores = new_graph.scores[e_mask]

In [ ]:
new_graph.edge_index.shape[1], new_graph.scores.shape[0]

In [ ]:
# Create a sparse matrix representation of the graph with edge scores
adj_matrix = to_scipy_sparse_matrix(new_graph.edge_index, num_nodes=new_graph.x.size(0))

In [ ]:
# Compute the connected components using SciPy
n_components, labels = scigraph.connected_components(csgraph=adj_matrix, directed=False, return_labels=True)

In [ ]:
labels

- Run Walkthrough

In [ ]:
import networkx as nx
from torch_geometric.utils import to_networkx

In [ ]:
# Convert to NetworkX DiGraph
G = to_networkx(new_graph, to_undirected=False)
G.remove_nodes_from(list(nx.isolates(G)))

In [ ]:
# CCL on NetworkX Graph ~ CCL on PyG Graph
# sparse_matrix = nx.to_scipy_sparse_matrix(G)
# n_components, labels = scigraph.connected_components(sparse_matrix, directed=False)

In [ ]:
n_components

In [ ]:
labels

In [ ]:
indices = np.where(labels == 0)[0]
indices

In [ ]:
subgraph = G.subgraph(indices)

In [ ]:
paths = []

In [ ]:
# get paths from components
for i in range(num_components):
    indices = np.where(labels == i)[0]
    subgraph = G.subgraph(indices)
    for source, target in subgraph.edges():
        for path in nx.all_simple_paths(subgraph, source=source, target=target):
            paths.append(path)

- Here I am trying to use your code to build the `track_df` similar to what I get from `eval/trkx_from_gnn.py` if we save this file then I can easily run `eval/eval_reco_trkx.py` that will output track efficiency, purity etc. I think I failed here. Or something is not right in builiding `paths` list.

In [ ]:
from itertools import chain

In [ ]:
track_df = pd.DataFrame(
    {
        "hit_id": list(chain.from_iterable(paths)),
        "track_id": list(chain.from_iterable([[i] * len(p) for i, p in enumerate(paths)])),
    }
)

# Remove duplicates on hit_id: TODO: In very near future, handle multiple tracks through the same hit!
track_df = track_df.drop_duplicates(subset="hit_id")

In [ ]:
hit_id = track_df.hit_id
track_id = track_df.track_id

track_id_tensor = torch.ones(len(graph.x), dtype=torch.long) * -1
track_id_tensor[hit_id.values] = torch.from_numpy(track_id.values)

new_graph.labels = track_id_tensor

In [ ]:
new_graph.labels

In [ ]:
# I dnot want to change rest of the code, re-assign
labeled_graph = new_graph

### _Plotting the CC_

In [ ]:
from src.drawing import detector_layout
from src.utils_math import polar_to_cartesian

- use `labeled_graph` for plotting

In [ ]:
# pids for true_track, unique labels for reco_tracks
true_track, reco_track = labeled_graph, labeled_graph

In [ ]:
# event id from the file itself
e_id = int(new_graph.event_file[-10:])
e_id

In [ ]:
r, phi, ir = labeled_graph.x.T
ir = ir.detach().numpy()*100
x, y = polar_to_cartesian(r.detach().numpy(), phi.detach().numpy())

- use `pid` to indentify true tracks.

In [ ]:
# plot true event
fig, ax = detector_layout(figsize=(8,8))
p_ids = np.unique(true_track.pid)

for pid in p_ids:
    idx = true_track.pid == pid
    ax.plot(x[idx], y[idx], '-', linewidth=1.5)
    ax.scatter(x[idx], y[idx], label='particle_id: {}'.format(int(pid)))

ax.set_title('Azimuthal View of STT, EventID # {}'.format(e_id))
ax.legend(fontsize=10, loc='best')
fig.tight_layout()
# fig.savefig("true_track.png")

- use `labels` as `pid` to identify reconstructed tracks.
- **problem might lie here**

In [ ]:
# plot reco event from CCL
fig, ax = detector_layout(figsize=(8,8))
t_ids = np.unique(reco_track.labels)

# here, (x,y,ir) comes from the true event,
# but idx comes from reco event from CCL

for tid in t_ids:
    idx = reco_track.labels == tid
    ax.plot(x[idx], y[idx], '-', linewidth=2)
    ax.scatter(x[idx], y[idx], label='particle_id: {}'.format(tid))
    
ax.set_title('Azimuthal View of STT, EventID # {}'.format(e_id))
ax.legend(fontsize=10, loc='best')
fig.tight_layout()
# fig.savefig("reco_track.png")

### Problem

Above Plotting works with the `new_graph.labels = labels` where labels comes from `connected-component()`, But now I have assinged labels from `paths` from NetworkX. The plotting works if there is a one-one relateion between `track_id, hit_id` i.e. in `track_df` all hits that belong to same track are given a unique id. One can see this plotting in the `stt4_seg.ipynb`. 

### _Three Ways to Produce Adjacency Matrix_
There are three ways to create a sparse matrix representation of the graph with edge scores

In [ ]:
input_file = gnn_files[event_idx]

In [ ]:
os.path.split(input_file)[0], os.path.split(input_file)[1]

In [ ]:
graph = torch.load(input_file, map_location=device)

In [ ]:
# resize scores to edge_index shape
scores = graph.scores[:graph.edge_index.shape[1]]

In [ ]:
e_mask = scores > 0.5   # size reducer
passing_edges = graph.edge_index[:, e_mask]
passing_scores = scores[e_mask]
num_nodes=graph.x.size(0)

In [ ]:
graph.edge_index.shape

In [ ]:
passing_edges.shape

In [ ]:
# (1) adjacency matrix
adj_matrix = to_scipy_sparse_matrix(
    passing_edges, num_nodes=graph.x.size(0)
)

In [ ]:
# (2) adjacency matrix
adj_matrix = sps.coo_matrix(
    (np.ones(passing_edges.shape[1]), passing_edges.cpu().numpy()),
    shape=(num_nodes, num_nodes),
)

In [ ]:
# (3) adjacency matrix
adj_matrix = sps.coo_matrix(
    (passing_scores.cpu().numpy(),
     (passing_edges.cpu().numpy())
     ),
    shape=(num_nodes, num_nodes)
)